In [108]:
#Some nice imports
import requests
import json
import numpy as np
import pandas as pd
from pandas import json_normalize

In [38]:
df= pd.read_csv('df_nyt_merged_cleaned.csv')

In [39]:
df.head()

,Unnamed: 0,rank,no_of_weeks,book,authors,publisher,description,date,author1,author2,nyt_name,good_reads_book,author_link,avg_rating,no_of_ratings,year,author_place,place,geometry.type,geometry.coordinates
0,0,1,New this week,TICK TOCK,James Patterson and Michael Ledwidge,"Little, Brown",The New York detective Michael Bennett enlists...,2011-02-13,James Patterson,Michael Ledwidge,TICK TOCK,"Tick Tock (Michael Bennett, #4)",https://www.goodreads.com/author/show/3780.Jam...,4.09,29129.0,2011,"Newburgh, New York, The United States","Newburgh, New York, The United States",Point,"[-74.0104178, 41.5034271]"
1,1,2,New this week,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,Knopf Doubleday,A hacker and a journalist investigate the disa...,2011-02-13,Stieg Larsson,NaN,THE GIRL WITH THE DRAGON TATTOO,"The Girl with the Dragon Tattoo (Millennium, #1)",https://www.goodreads.com/author/show/706255.S...,4.16,3108073.0,2011,"Skelleftehamn , Sweden","Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
2,2,3,New this week,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,Knopf Doubleday,In the second volume of the Millennium trilogy...,2011-02-13,Stieg Larsson,NaN,THE GIRL WHO PLAYED WITH FIRE,"The Girl Who Played with Fire (Millennium, #2)",https://www.goodreads.com/author/show/706255.S...,4.25,909723.0,2011,"Skelleftehamn , Sweden","Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
3,3,4,New this week,THE GIRL WHO KICKED THE HORNET’S NEST,Stieg Larsson,Knopf Doubleday,"The third volume of the Millennium trilogy, ab...",2011-02-13,Stieg Larsson,NaN,THE GIRL WHO KICKED THE HORNET’S NEST,The Girl Who Kicked the Hornet's Nest (Millenn...,https://www.goodreads.com/author/show/706255.S...,4.23,725317.0,2011,"Skelleftehamn , Sweden","Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
4,4,5,New this week,WATER FOR ELEPHANTS,Sara Gruen,Algonquin,"After his parents die in a car accident, young...",2011-02-13,Sara Gruen,NaN,WATER FOR ELEPHANTS,Water for Elephants,https://www.goodreads.com/author/show/24556.Sa...,4.11,1586438.0,2011,"Vancouver, Canada","Vancouver, Canada",Point,"[-123.1207375, 49.2827291]"


In [61]:
# keeping only the columns needed for the map

df_map = df[['author1', 'year', 'avg_rating', 'no_of_ratings', 'place','geometry.type', 'geometry.coordinates']]

In [62]:
df_map.drop_duplicates()
df_map.dropna(subset=['avg_rating', 'no_of_ratings', 'geometry.coordinates'], inplace=True)

/var/folders/bp/pstyf_r14hx35s64y_db5pq40000gn/T/ipykernel_10491/3077366406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_map.dropna(subset=['avg_rating', 'no_of_ratings', 'geometry.coordinates'], inplace=True)


In [63]:
df_stats = df_map.groupby(['year', 'author1']).agg({'avg_rating': 'mean', 'no_of_ratings': 'sum'})

In [64]:
df_map

,author1,year,avg_rating,no_of_ratings,place,geometry.type,geometry.coordinates
0,James Patterson,2011,4.09,29129.0,"Newburgh, New York, The United States",Point,"[-74.0104178, 41.5034271]"
1,Stieg Larsson,2011,4.16,3108073.0,"Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
2,Stieg Larsson,2011,4.25,909723.0,"Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
3,Stieg Larsson,2011,4.23,725317.0,"Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
4,Sara Gruen,2011,4.11,1586438.0,"Vancouver, Canada",Point,"[-123.1207375, 49.2827291]"
...,...,...,...,...,...,...,...
9981,Patrick Rothfuss,2023,4.08,4919.0,"Madison, Wisconsin, The United States",Point,"[-89.4007501, 43.0721661]"
9983,Michael Connelly,2023,4.65,16198.0,"Philadelphia, Pennsylvania, The United States",Point,"[-75.1652215, 39.9525839]"
9984,Anthony Doerr,2023,4.32,1551610.0,"Cleveland, Ohio, The United States",Point,"[-81.6943605, 41.49932]"
9985,Bonnie Garmus,2023,4.31,963010.0,"Riverside, CA, The United States",Point,"[-117.3754942, 33.9806005]"


In [65]:
df_map_merged = df_stats.merge(df_map, left_on=['author1','year'], right_on=['author1','year'], how='left')



In [58]:
df_map_merged

,author1,year,avg_rating_x,no_of_ratings_x,avg_rating_y,no_of_ratings_y,geometry.type,geometry.coordinates
0,Aaron Allston,2011,3.890,5100.0,3.89,5100.0,Point,"[-96.46887269999999, 32.0954304]"
1,Alexander McCall Smith,2011,4.090,16886.0,4.09,16886.0,Point,"[28.5873388, -20.1457125]"
2,Amanda Quick,2011,4.090,9236.0,4.09,9236.0,Point,"[-95.712891, 37.09024]"
3,Ann Brashares,2011,3.910,37054.0,3.91,37054.0,Point,"[-78.6568942, 37.4315734]"
4,Ann Patchett,2011,3.890,1299648.0,3.89,185664.0,Point,"[-118.242643, 34.0549076]"
...,...,...,...,...,...,...,...,...
7212,Tessa Bailey,2023,3.595,140686.0,3.32,88104.0,Point,"[-117.3794834, 33.1958696]"
7213,Tessa Bailey,2023,3.595,140686.0,3.87,52582.0,Point,"[-117.3794834, 33.1958696]"
7214,Tom Hanks,2023,3.720,10408.0,3.72,10408.0,Point,"[-122.0310733, 37.9779776]"
7215,V.E. Schwab,2023,4.420,11248.0,4.42,11248.0,Point,"[-95.712891, 37.09024]"


In [66]:
df_map_merged.drop(columns=['avg_rating_x', 'no_of_ratings_x'])

,author1,year,avg_rating_y,no_of_ratings_y,place,geometry.type,geometry.coordinates
0,Aaron Allston,2011,3.89,5100.0,"Corsicana, Texas, The United States",Point,"[-96.46887269999999, 32.0954304]"
1,Alexander McCall Smith,2011,4.09,16886.0,Bulawayo (formerly the British colony of South...,Point,"[28.5873388, -20.1457125]"
2,Amanda Quick,2011,4.09,9236.0,The United States,Point,"[-95.712891, 37.09024]"
3,Ann Brashares,2011,3.91,37054.0,"Virginia, The United States",Point,"[-78.6568942, 37.4315734]"
4,Ann Patchett,2011,3.89,185664.0,"Los Angeles, CA, The United States",Point,"[-118.242643, 34.0549076]"
...,...,...,...,...,...,...,...
7212,Tessa Bailey,2023,3.32,88104.0,"Oceanside, California, The United States",Point,"[-117.3794834, 33.1958696]"
7213,Tessa Bailey,2023,3.87,52582.0,"Oceanside, California, The United States",Point,"[-117.3794834, 33.1958696]"
7214,Tom Hanks,2023,3.72,10408.0,"Concord, California, The United States",Point,"[-122.0310733, 37.9779776]"
7215,V.E. Schwab,2023,4.42,11248.0,The United States,Point,"[-95.712891, 37.09024]"


##### Building the prperties for the map

**name:** This is the name of the location that shows up when you rollover the shape.

In [152]:
df_map_merged['properties.name'] = df_map_merged['place'].apply(lambda x: x.title())

**headline:** This is the lead sentence or bullet point displayed when you rollover the shape.

In [153]:
df_map_merged['properties.headline'] =df_map_merged['author1']

**article:** I want to mention the no. of Goodreads ratings this suthor got for that year

In [183]:
def rating_text(ratings):
    return f" from {ratings: ,.0f} ratings on Goodreads"

def average_text(avg) :
    return f"Average book rating of {avg: ,.1f} "

In [184]:
df_map_merged['properties.article'] = df_map_merged['avg_rating_y'].apply(average_text) + df_map_merged['no_of_ratings_y'].apply(rating_text)

In [188]:
df_map_merged.tail()

,author1,year,avg_rating_x,no_of_ratings_x,avg_rating_y,no_of_ratings_y,place,geometry.type,geometry.coordinates,properties.name,properties.headline,properties.article,properties.group_id,properties.group_name,properties.color,geometry_coordinates_old
7212,Tessa Bailey,2023,3.595,140686.0,3.32,88104.0,"Oceanside, California, The United States",Point,"[-117.3794834, 33.1958696]","Oceanside, California, The United States",Tessa Bailey,"Average book rating of 3.3 from 88,104 rati...",13,2023,#D482EF,"[-117.3794834, 33.1958696]"
7213,Tessa Bailey,2023,3.595,140686.0,3.87,52582.0,"Oceanside, California, The United States",Point,"[-117.3794834, 33.1958696]","Oceanside, California, The United States",Tessa Bailey,"Average book rating of 3.9 from 52,582 rati...",13,2023,#D482EF,"[-117.3794834, 33.1958696]"
7214,Tom Hanks,2023,3.720,10408.0,3.72,10408.0,"Concord, California, The United States",Point,"[-122.0310733, 37.9779776]","Concord, California, The United States",Tom Hanks,"Average book rating of 3.7 from 10,408 rati...",13,2023,#D482EF,"[-122.0310733, 37.9779776]"
7215,V.E. Schwab,2023,4.420,11248.0,4.42,11248.0,The United States,Point,"[-95.712891, 37.09024]",The United States,V.E. Schwab,"Average book rating of 4.4 from 11,248 rati...",13,2023,#AB0AEF,"[-95.712891, 37.09024]"
7216,William Kent Krueger,2023,4.370,22331.0,4.37,22331.0,"Torrington, Wyoming, The United States",Point,"[-104.1843942, 42.0624646]","Torrington, Wyoming, The United States",William Kent Krueger,"Average book rating of 4.4 from 22,331 rati...",13,2023,#AB0AEF,"[-104.1843942, 42.0624646]"


**group_id:** This separates different groups with the pulldown menu. Showing everything should be group 0, Individual groups should begin at 1 And go up in order (2, 3, 4). If you want to have completely different groups showing different information but in the same place, talk to me about that.

**group_name:** should correspond to **group_id:** It is the name that shows up in the menubar.

In [189]:
df_map_merged['properties.group_id'] = pd.factorize(df_map_merged['year'])[0] + 1

In [190]:
df_map_merged['properties.group_name'] = df_map_merged['year']

**color:** This will set the color for every shape. Here we are making semi-random colors for every single shape...Not a good thing to do. But definitely have a lot of funWith your color algorithms here. You want the colors to reflect different ranges of values. Random is the last thing you want to do. But the function below builds random hexadecimal color values.

In [191]:
def color_ramp(value):
    if value < 3 :
        hexColor = '#D8C7F0'
    elif (value > 3) and (value < 3.5) :
        hexColor = '#DAABF0'
    elif (value> 3.5) and (value < 4) :
        hexColor = '#D482EF'
    elif value > 4 and value < 4.5 :
        hexColor = '#AB0AEF'
    elif value > 4.5 :
        hexColor = '#7B23EF'
    else :
        hexColor = '#FFFFFF'


    return hexColor

In [192]:
df_map_merged['properties.color'] = df_map_merged['avg_rating_x'].apply(lambda x: color_ramp(x))

In [193]:
df_map_merged

,author1,year,avg_rating_x,no_of_ratings_x,avg_rating_y,no_of_ratings_y,place,geometry.type,geometry.coordinates,properties.name,properties.headline,properties.article,properties.group_id,properties.group_name,properties.color,geometry_coordinates_old
0,Aaron Allston,2011,3.890,5100.0,3.89,5100.0,"Corsicana, Texas, The United States",Point,"[-96.46887269999999, 32.0954304]","Corsicana, Texas, The United States",Aaron Allston,"Average book rating of 3.9 from 5,100 ratin...",1,2011,#D482EF,"[-96.46887269999999, 32.0954304]"
1,Alexander McCall Smith,2011,4.090,16886.0,4.09,16886.0,Bulawayo (formerly the British colony of South...,Point,"[28.5873388, -20.1457125]",Bulawayo (Formerly The British Colony Of South...,Alexander McCall Smith,"Average book rating of 4.1 from 16,886 rati...",1,2011,#AB0AEF,"[28.5873388, -20.1457125]"
2,Amanda Quick,2011,4.090,9236.0,4.09,9236.0,The United States,Point,"[-95.712891, 37.09024]",The United States,Amanda Quick,"Average book rating of 4.1 from 9,236 ratin...",1,2011,#AB0AEF,"[-95.712891, 37.09024]"
3,Ann Brashares,2011,3.910,37054.0,3.91,37054.0,"Virginia, The United States",Point,"[-78.6568942, 37.4315734]","Virginia, The United States",Ann Brashares,"Average book rating of 3.9 from 37,054 rati...",1,2011,#D482EF,"[-78.6568942, 37.4315734]"
4,Ann Patchett,2011,3.890,1299648.0,3.89,185664.0,"Los Angeles, CA, The United States",Point,"[-118.242643, 34.0549076]","Los Angeles, Ca, The United States",Ann Patchett,"Average book rating of 3.9 from 185,664 rat...",1,2011,#D482EF,"[-118.242643, 34.0549076]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7212,Tessa Bailey,2023,3.595,140686.0,3.32,88104.0,"Oceanside, California, The United States",Point,"[-117.3794834, 33.1958696]","Oceanside, California, The United States",Tessa Bailey,"Average book rating of 3.3 from 88,104 rati...",13,2023,#D482EF,"[-117.3794834, 33.1958696]"
7213,Tessa Bailey,2023,3.595,140686.0,3.87,52582.0,"Oceanside, California, The United States",Point,"[-117.3794834, 33.1958696]","Oceanside, California, The United States",Tessa Bailey,"Average book rating of 3.9 from 52,582 rati...",13,2023,#D482EF,"[-117.3794834, 33.1958696]"
7214,Tom Hanks,2023,3.720,10408.0,3.72,10408.0,"Concord, California, The United States",Point,"[-122.0310733, 37.9779776]","Concord, California, The United States",Tom Hanks,"Average book rating of 3.7 from 10,408 rati...",13,2023,#D482EF,"[-122.0310733, 37.9779776]"
7215,V.E. Schwab,2023,4.420,11248.0,4.42,11248.0,The United States,Point,"[-95.712891, 37.09024]",The United States,V.E. Schwab,"Average book rating of 4.4 from 11,248 rati...",13,2023,#AB0AEF,"[-95.712891, 37.09024]"


In [167]:
# #df_map_merged['geometry_coordinates_old']= df_map_merged['geometry.coordinates']
# df_map_merged['geometry.coordinates'] = df_map_merged['geometry.coordinates'].apply(lambda coords: eval(coords))

TypeError: eval() arg 1 must be a string, bytes or code object

In [194]:
ok_json = json.loads(df_map_merged.to_json(orient='records'))

In [195]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data


In [196]:
geo_format = process_to_geojson(ok_json)

In [197]:
geo_format

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Corsicana, Texas, The United States',
    'headline': 'Aaron Allston',
    'article': 'Average book rating of  3.9  from  5,100 ratings on Goodreads',
    'group_id': 1,
    'group_name': 2011,
    'color': '#D482EF'},
   'geometry': {'type': 'Point', 'coordinates': [-96.4688727, 32.0954304]}},
  {'type': 'Feature',
   'properties': {'name': 'Bulawayo (Formerly The British Colony Of Southern Rhodesia), Zimbabwe',
    'headline': 'Alexander McCall Smith',
    'article': 'Average book rating of  4.1  from  16,886 ratings on Goodreads',
    'group_id': 1,
    'group_name': 2011,
    'color': '#AB0AEF'},
   'geometry': {'type': 'Point', 'coordinates': [28.5873388, -20.1457125]}},
  {'type': 'Feature',
   'properties': {'name': 'The United States',
    'headline': 'Amanda Quick',
    'article': 'Average book rating of  4.1  from  9,236 ratings on Goodreads',
    'group_id': 1,
    'group_name': 2011,


In [198]:
# export this file
#Variable name
with open('geo-data-goodreads.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data-goodreads.js', 'a') as outfile:
    json.dump(geo_format, outfile)